# 3DEP Seamless Elevation → H3 Hexagons

Concurrent pipeline: query USGS 3DEP seamless DEM via STAC (Planetary Computer) →
process tiles with odc-stac → aggregate to H3 hexagons via DuckDB → render with lonboard.

**Run with:** `uvx juv run elevation_h3_clean.ipynb`

In [ ]:
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "duckdb==1.4.3",
#     "h3>=4.0.0",
#     "lonboard==0.13.0",
#     "matplotlib==3.10.8",
#     "morecantile>=1.0.0",
#     "numpy==2.2.0",
#     "odc-stac==0.5.0",
#     "palettable==3.3.3",
#     "planetary-computer==1.0.0",
#     "pyarrow==18.1.0",
#     "pyproj==3.7.2",
#     "pystac-client==0.9.0"
# ]
# ///

In [ ]:
import numpy as np
import pyarrow as pa
import duckdb
import morecantile
import h3
import odc.stac
import planetary_computer
import pystac_client
from concurrent.futures import ThreadPoolExecutor, as_completed
from matplotlib.colors import Normalize
from palettable.scientific.sequential import Batlow_20
from pyproj import Transformer
from arro3.core import Table

from lonboard import Map, H3HexagonLayer
from lonboard.colormap import apply_continuous_cmap

import warnings
warnings.filterwarnings("ignore", message="Dataset has no geotransform", category=UserWarning)

## Pipeline Functions

In [ ]:
def calculate_resolution_for_h3(h3_res, native_resolution=10, pixels_per_hex_edge=6):
    """Calculate odc-stac resolution to get ~pixels_per_hex_edge pixels per H3 hex edge.

    For COGs, resolution controls which internal overview GDAL reads:
    coarser resolution = smaller overview = less I/O.
    """
    hex_edge_m = h3.average_hexagon_edge_length(h3_res, unit='m')
    target = hex_edge_m / pixels_per_hex_edge
    resolution = max(round(target / native_resolution) * native_resolution, native_resolution)
    px_per_edge = hex_edge_m / resolution
    print(f"H3 res {h3_res}: hex edge {hex_edge_m:.0f}m, resolution {resolution}m, {px_per_edge:.1f} px/edge")
    return resolution


def query_stac(bbox, collection):
    """Query Planetary Computer STAC catalog for items covering bbox."""
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    items = catalog.search(
        collections=[collection],
        bbox=bbox,
        query={"gsd": {"eq": 10}}
    ).item_collection()
    print(f"Found {len(items)} STAC items")
    return items


def get_tiles(bbox, zoom):
    """Split bbox into morecantile tiles at given zoom level."""
    tms = morecantile.tms.get("WebMercatorQuad")
    tiles = list(tms.tiles(*bbox, zooms=[zoom]))
    print(f"{len(tiles)} tiles at zoom {zoom}")
    return tiles, tms


duckdb.sql("INSTALL h3 FROM community")


def get_con():
    """In-memory connection for workers. LOAD only, no INSTALL."""
    con = duckdb.connect()
    con.sql("""
        SET temp_directory = './tmp';
        SET memory_limit = '512MB';
        LOAD h3;
    """)
    return con


def process_tile_to_h3(tile, tms, items, band, h3_res, resolution):
    """Load one tile's DEM, reproject to 4326, aggregate to H3.

    Returns Arrow table (hex, metric) or None on failure.
    """
    tile_bounds = tms.bounds(tile)
    tile_bbox = [tile_bounds.left, tile_bounds.bottom, tile_bounds.right, tile_bounds.top]
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

    try:
        ds = odc.stac.load(
            items,
            crs="EPSG:3857",
            resolution=resolution,
            bands=[band],
            bbox=tile_bbox,
        ).astype(float)
    except Exception:
        return None

    arr = ds[band].max(dim="time")
    vals = arr.values
    x_coords = arr.coords["x"].values
    y_coords = arr.coords["y"].values
    X, Y = np.meshgrid(x_coords, y_coords)
    lons, lats = transformer.transform(X.flatten(), Y.flatten())

    tile_pa = pa.table({
        "lat": pa.array(lats, type=pa.float64()),
        "lng": pa.array(lons, type=pa.float64()),
        "elevation": pa.array(vals.flatten(), type=pa.float64()),
    })

    con = get_con()
    result = con.sql(f"""
        SELECT
            h3_latlng_to_cell_string(lat, lng, {h3_res}) AS hex,
            AVG(elevation) AS metric
        FROM tile_pa
        GROUP BY 1
    """).fetch_arrow_table()
    return result


def process_all_tiles(items, tiles, tms, band, h3_res, resolution, max_workers=4):
    """Process all tiles concurrently, then merge edge hexagons."""
    batches = []
    completed = 0
    total = len(tiles)

    with ThreadPoolExecutor(max_workers=max_workers) as pool:
        futures = {
            pool.submit(process_tile_to_h3, tile, tms, items, band, h3_res, resolution): tile
            for tile in tiles
        }
        for future in as_completed(futures):
            result = future.result()
            if result is not None and len(result) > 0:
                batches.append(result)
            completed += 1
            if completed % 100 == 0 or completed == total:
                print(f"  Processed {completed}/{total} tiles")

    if not batches:
        raise RuntimeError("No tiles produced data")

    combined = pa.concat_tables(batches)
    print(f"Pre-merge hex count: {len(combined):,}")

    con = duckdb.connect()
    hex_result = con.sql("""
        SELECT hex, AVG(metric) AS metric
        FROM combined
        GROUP BY 1
    """).fetch_arrow_table()
    con.close()
    print(f"Final H3 hexagons: {len(hex_result):,}")
    return hex_result

## Configuration

Pick a bounding box and H3 resolution. Get bbox coordinates from
[Bounding Box Tool](https://boundingbox.klokantech.com/) — use **CSV** format (west, south, east, north).

Browse colormaps at the [palettable gallery](https://jiffyclub.github.io/palettable/).

In [ ]:
# Mount Washington, NH — dramatic terrain, reasonable size
bbox = [-71.502182, 44.092909, -70.723358, 44.511611]

# Grand Canyon (large — expect ~1M+ hexes at res 11, may take several minutes)
# bbox = [-113.0606, 35.8461, -111.7165, 36.7665]

# Cairo, IL (Mississippi/Ohio confluence)
# bbox = [-89.4436, 36.91, -89.0463, 37.1834]

# Greenville, MS (Mississippi Delta)
# bbox = [-91.3756, 32.447, -90.859, 33.9034]

COLLECTION = "3dep-seamless"
BAND = "data"
H3_RES = 11
NATIVE_RESOLUTION = 10  # 3DEP seamless native pixel size (meters)
RESOLUTION = calculate_resolution_for_h3(H3_RES, NATIVE_RESOLUTION)
TILE_ZOOM = 12
MAX_WORKERS = 8

## Run Pipeline

In [ ]:
items = query_stac(bbox, COLLECTION)
tiles, tms = get_tiles(bbox, TILE_ZOOM)
hex_result = process_all_tiles(items, tiles, tms, BAND, H3_RES, RESOLUTION, max_workers=MAX_WORKERS)

## Visualize

In [ ]:
table = Table.from_arrow(hex_result)
del hex_result  # free PyArrow copy

elev_values = table["metric"].to_pylist()
normalizer = Normalize(min(elev_values), max(elev_values))
colors = apply_continuous_cmap(normalizer(elev_values), Batlow_20, alpha=1)

layer = H3HexagonLayer(
    table=table,
    get_hexagon=table["hex"],
    get_fill_color=colors,
    high_precision=True,
    stroked=False,
    get_elevation=table["metric"],
    extruded=True,
    elevation_scale=3.4,
    opacity=0.9,
)

view_state = {
    "longitude": (bbox[0] + bbox[2]) / 2,
    "latitude": (bbox[1] + bbox[3]) / 2,
    "zoom": 10,
    "pitch": 55,
    "bearing": -20,
}

m = Map(layers=[layer], view_state=view_state)
m